In [7]:
import numpy as np
import pandas as pd
import time
import tkinter as tk
from openpyxl import load_workbook
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from tabulate import tabulate
from webdriver_manager.chrome import ChromeDriverManager
servico = Service(ChromeDriverManager().install())
import funcoes as fc
import pyautogui

In [11]:
reset = True
while reset:
    navegador = fc.abreChromeEdge()
    while True:
        #INICIA JANELA
        try:
            fc.iniciaJanela(navegador)
            user_name = fc.user_name
        #CONDICAO DE ERRO PARA CASO O USUÁRIO ERRE O SEU LOGIN 
        except Exception:
            print('Ocorreu um erro, tente novamente.')
        try:
            #FECHA ALERTA DO NAVEGADOR
            time.sleep(1)
            alert = Alert(navegador)
            alert.accept()
        except:
            #ENTRAR NA CAIXA DE PROCESSOS ATRIBUIDOS AO USUARIO
            if fc.check_element_exists(By.XPATH,'//*[@id="divFiltro"]/div[2]/a', navegador):
                navegador.find_element(By.XPATH,'//*[@id="divFiltro"]/div[2]/a').click()
                break
            #CASO NAO ENCONTRE O FILTRO DE PROCESSOS ELE INFORMA QUE NAO ENTROU NO SEI
            else:
                #APAGA O USUÁRIO
                navegador.find_element(By.XPATH,'//*[@id="txtUsuario"]').clear()
                print('Usuário ou senha incorretos. Digite novamente')
    
    #PEGA O CORPO DA TABELA NO NAVEGADOR
    tbody = navegador.find_element(By.XPATH, '//*[@id="tblProcessosRecebidos"]/tbody')
    #PEGA TODAS AS LINHAS (TR) DO CORPO DA TABELA, EXCETO A PRIMEIRA
    trs = tbody.find_elements(By.TAG_NAME, 'tr')[1:]

    #CRIA UM EXECUTOR DE THREADS
    executor = ThreadPoolExecutor()
    #APLICA A FUNÇÃO 'PROCESSA_TR' A CADA LINHA (TR) DA TABELA USANDO MULTITHREADING
    resultados = list(executor.map(fc.processa_tr, trs))
    #AGUARDA TODAS AS THREADS TERMINAREM
    executor.shutdown(wait=True)

    #CRIA DUAS LISTAS PARA ARMAZENAR OS PROCESSOS QUE SERÃO ANALISADOS E CONCLUÍDOS
    lista_processos = [r[0] for r in resultados if not r[1]]
    lista_procConformes = [r[0] for r in resultados if r[1] and r[2]]

    #INVERTE A ORDEM DOS PROCESSOS NAS LISTAS
    lista_processos.reverse()

    #IMPRIME A QUANTIDADE DE PROCESSOS QUE SERÃO ANALISADOS E CONCLUÍDOS
    print("Quantidade de processos para analisar",len(lista_processos))
    print("Quantidade de processos para concluir", len(lista_procConformes))

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF79B6E3AF5+28005]
	(No symbol) [0x00007FF79B6483F0]
	(No symbol) [0x00007FF79B4E580A]
	(No symbol) [0x00007FF79B4BFA85]
	(No symbol) [0x00007FF79B562AD7]
	(No symbol) [0x00007FF79B57B1B1]
	(No symbol) [0x00007FF79B55B7E3]
	(No symbol) [0x00007FF79B5275C8]
	(No symbol) [0x00007FF79B528731]
	GetHandleVerifier [0x00007FF79B9D646D+3118813]
	GetHandleVerifier [0x00007FF79BA26CC0+3448624]
	GetHandleVerifier [0x00007FF79BA1CF3D+3408301]
	GetHandleVerifier [0x00007FF79B7AA44B+841403]
	(No symbol) [0x00007FF79B65344F]
	(No symbol) [0x00007FF79B64F4C4]
	(No symbol) [0x00007FF79B64F65D]
	(No symbol) [0x00007FF79B63EBB9]
	BaseThreadInitThunk [0x00007FF9E2F0257D+29]
	RtlUserThreadStart [0x00007FF9E3CEAF08+40]
